In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime, random
import matplotlib.dates as mdates

In [ ]:
def MakeDataFromText(filepath):
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif len(line)==8:               # If it is a time stamp
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1970, 2, day, int(h) - 3, int(m), int(s))) # Minus 3 for Pacific Time
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db

class Person:
    """
    Class to represent each individal poutfit being recorded during Deployment, made up of garments.
    """
    def __init__(
        self, wet:bool, Name:str, color:str,
        backneck:str,   back:str,        chest:str, 
        leftab:str,     rightab:str, 
        rightuparm:str, rightlowarm:str, 
        leftuparm:str,  leftlowarm:str, 
        beanie1:str,    beanie2:str,     beanie3:str, beanieR:str, beanieL:str
    ):
        self.Backneck    = backneck        # Tag ID's for the shirt
        self.Back        = back
        self.Chest       = chest
        self.Leftab      = leftab
        self.Rightab     = rightab
        self.Rightuparm  = rightuparm
        self.Rightlowarm = rightlowarm
        self.Leftuparm   = leftuparm
        self.Leftlowarm  = leftlowarm
        self.Beanie1     = beanie1         # Tag ID's for the beanies
        self.Beanie2     = beanie2
        self.Beanie3     = beanie3
        self.BeanieR     = beanieR
        self.BeanieL     = beanieL

        self.Name = Name                   # Soldier's Name

        self.ShirtMapping = {
            self.Backneck:    Name + " Back Neck",
            self.Back:        Name + " Back",
            self.Chest:       Name + " Chest",
            self.Leftab:      Name + " Left Abdomen",
            self.Rightab:     Name + " Right Abdomen",
            self.Rightuparm:  Name + " Right Upper Arm",
            self.Rightlowarm: Name + " Right Lower Arm",
            self.Leftlowarm:  Name + " Left Lower Arm",
            self.Leftuparm:   Name + " Left Upper Arm",
        }

        self.TagList = [
            self.Backneck,   self.Back, self.Chest,
            self.Leftab,     self.Rightab,
            self.Rightuparm, self.Rightlowarm,
            self.Leftuparm,  self.Leftlowarm,
            self.Beanie1,    self.Beanie2, self.Beanie3, self.BeanieR, self.BeanieL
        ]

        self.BeanieList = [self.Beanie1, self.Beanie2, self.Beanie3, self.BeanieR, self.BeanieL]
        self.Color = color

        self.wet = wet # Bool for wetness check
        if wet:
            self.BackColor = "#870101"
            self.BackNeckColor = "#ad4503"
            self.ChestColor = "#756a01"
            self.LeftAbColor = "#396102"
            self.RightAbColor = "#015c0b"
            self.LeftUpArmColor = "#01663f"
            self.RightUpArmColor = "#00a191"
            self.LeftLowArmColor = "#012173"
            self.RightLowArmColor = "#1e075e"
            
        else:
            self.BackColor = "#ff0d0d"
            self.BackNeckColor = "#ff7214"
            self.ChestColor = "#d1bc02"
            self.LeftAbColor = "#6bb803"
            self.RightAbColor = "#01bf16"
            self.LeftUpArmColor = "#03c278"
            self.RightUpArmColor = "#06b4cf"
            self.LeftLowArmColor = "#0443e0"
            self.RightLowArmColor = "#4c05e6"

In [ ]:
# Paul P, only wet
person6  = Person(True, "Paul P.", "#de0413", "71BB", "7705", "B25E", "3247", "A9B5", "6C38", "7662", "A983", "098F", "B644", "7BCC", "B576", "70D1", "4D84");

# Joelle, only wet
person14 = Person(True, "Joelle", "#9e05a1", "462C", "5B60", "7415", "6310", "1851", "616A", "5DAA", "6D28", "9991", "3B0E", "5374", "682C", "0A50", "1543");

# Garrett C, only wet
person28 = Person(True, "Garrett C.", "#0909eb", "8977", "4EB3", "AA5E", "6626", "9ABE", "6CD0", "AB8E", "14AA", "6E94", "895D", "571F", "2D68", "8121", "198C");

# Dennis, only wet
person2  = Person(True, "Dennis", "#078c26", "333B", "289B", "2473", "231D", "9879", "4067", "5FB6", "169E", "8D50", "74C4", "73DC", "DC4A", "884D", "1BA4");

# Brent, only wet
person17 = Person(True, "Brent L.", "#e8c100", "6CC7", "8FB7", "799D", "6F8D", "6332", "5A1E", "92A5", "4A3D", "3EC9", "16AB", "A66C", "6162", "A686", "7E1B");

# Jackson, wet and dry
person15 = Person(True,  "Jackson Wet", "#0c008f", "9C54", "9968", "65E1", "5E23", "ACB3", "7AD3", "99A1", "B19D", "43C6", "7DCB", "9D35", "6FD0", "34CE", "62E3");
person11 = Person(False, "Jackson Dry", "#3845ff", "2A1B", "238C", "731F", "9591", "5C98", "5F06", "4526", "461C", "5253", "B078", "3AC7", "566B", "65D4", "432B");

# Matt C, wet and dry
person16 = Person(True,  "Matt C. Wet", "#02591e", "959E", "1C5F", "5A59", "077A", "902A", "3B60", "8199", "4469", "813A", "B7A4", "5163", "4DA5", "8995", "7972");
person8  = Person(False, "Matt C. Dry", "#00ed4f", "1B6A", "0D42", "7AD4", "20AF", "493F", "404A", "6878", "1A3B", "546F", "194B", "3133", "A847", "0D50", "7A61");

# William D, wet and dry
person13 = Person(True,  "William D. Wet", "#a60000", "A033", "A0C0", "892F", "627F", "6BD9", "3DB0", "8C97", "4ECF", "A73C", "90AE", "78D3", "82D8", "312D", "2A29");
person10 = Person(False, "William D. Dry", "#ff3037", "366C", "A08B", "AC42", "9AC9", "B53F", "76A4", "5E76", "68AE", "41D6", "803F", "5757", "9E54", "----", "----");

# Sean, wet and dry
person32 = Person(True,  "Sean Wet", "#7d0080", "2990", "1EC3", "51E2", "3783", "AE5E", "4575", "0688", "A0B0", "1BA6", "3C3E", "1989", "7C2C", "42D7", "6129");
person31 = Person(False, "Sean Dry", "#fb24ff", "1CAE", "7764", "8F24", "AD3E", "8898", "413F", "097A", "18A8", "4258", "7F4E", "801E", "8651", "AAB4", "6CB5");

# Harold, wet and dry
person19 = Person(True,  "Harold Wet", "#003952", "697F", "78A5", "5D0E", "7EC6", "AE59", "8158", "4A9B", "1D44", "2122", "5C02", "2D90", "6DAB", "3374", "B1AB");
person21 = Person(False, "Harold Dry", "#008dc9", "8CB4", "4115", "4D74", "4883", "4C9F", "ABB7", "7938", "390F", "21A5", "A697", "7C7D", "B689", "114D", "047A");

# Katherina, wet and dry
person26 = Person(True,  "Katherina Wet", "#965500", "4813", "2791", "1F7A", "6F4A", "2383", "2F66", "2E5C", "926A", "2E56", "158F", "5B06", "B28A", "73C6", "819C");
person22 = Person(False, "Katherina Dry", "#ed8600", "4F0D", "90D5", "0874", "45CE", "6CAC", "3EB2", "A26F", "888C", "50C3", "3767", "664F", "36A4", "2759", "BB8B");

In [ ]:
### BASELINE MEASUREMENTS ###
bl_back = [28.65]*101 + [20.30] + [31.50]*44 + [28.06]*120 + [25.56]*87 + [31.14]*137 + [32.43]*417+[27.34]*126 + [32.0]*223 +[32.22]*100 +[29.22]*31
bl_neck = 11*[35.89] + [32.51]*12+[31.61]*6 + 38*[28.89]+[31.09]*91 + [34.88]*25 + [20.25]* 295 + [18.8]*3 + [31.45]*19 + [28.02]*16+17*[28.43]
bl_lup = [26.59]*264 + [25.42]* 444 + [30.62]*477 + [30.45]* 513 + [27.77]* 653 + [29.21]*257 + [28.22]*263+388*[31.09]+731*[27.22] + 1288*[31.48]
bl_low = 52*[26.35]+[26.01]* 151 + 232*[28.56]+[25.15]*257+[ 30.23]* 117 + [30.75]*93 + [23.05 ]*83 + [30.19]*4 + [31.81]*29+165*[33.07] + [30.13]*191
bl_rup = 275 * [28.66]+398*[29.42]+ [24.96]*242 + [28.02]*540 + [31.46]*307 + [32.56]* 492 + [27.78]*211+[29.88]*33 + [32.49]*392 + 1240*[33.36]
bl_row =114*[23.84]+ [30.71]* 11 + [26.73]* 307 + [31.7]* 45 + [31.42]* 384 + [20.09]* 85 + [30.06]*23 + [31.10]*116+13*[29.35]+232*[27.8]
bl_lab =68*[30.43]+ 121*[32.12]+[22.18]* 86 +[ 31.44]* 344 + [31.08 ]*26 + [28.07 ]*463+[26.05]*170+[33.66]*5 + [29.29]*85 + [28.92]*145+[24.36]*438
bl_rab = 44 * [22.23]+[29.03]* 59 + 576*[34.54] +245*[24.47]+[32.94]* 307 + [25.41]*111 + [28.97]* 195 +[24.84 ]*416+[22.90]*4 + [28.57]*113 + [24.49]*419
bl_chst = 10*[27.53]+[28.91]*95+1489*[29.38] +430*[24.8]+ [24.9]* 244 + [27.6]*511+[29.67]*250+[26.0]* 617 + 25*[25.77] + [28.38]*235+[27.24]*321
bl_b1 =10*[29.6]+ [30.10]*25+ [26.41]*78 + [25.35]*17+[31.45]*32 + [32.09]*63 + [27.68]*255 +  [24.63]* 261 + [27.67]*28
bl_b2 =[31.63]* 85 + [26.46]*23 + [32.31]*42 + [25.64]*161 + [24.25]*279 + [19.40] + [29.22]*91 + 20*[29.46] + [27.27]*6+187*[29.45]
bl_b3 =[25.92]*7 + [27.74]*76 + [25.8]* 245 + [29.40] + [28.9]*7 + 30*[29.59] + 18 * [28.57]
bl_bL =[29.7]*62+[28.12]*12+[30.63]*70+[31.73]* 20 + [28.24]*92 + 5*[22.87] + [32.04]*10 + 105*[29.43]
bl_bR =[31.96]*27 + [28.11 ]*81 + [23.08]* 276 + [25.68]*45 + [32.79]*39 + 163 * [30.14]

In [ ]:
# print("Back: ", round(np.mean(bl_back),2), round(np.std(bl_back),2), len(bl_back))
# print("Neck: ", round(np.mean(bl_neck),2), round(np.std(bl_neck),2), len(bl_neck))
# print("Left Upper Arm: ", round(np.mean(bl_lup),2), round(np.std(bl_lup),2), len(bl_lup))
# print("Left Lower Arm: ", round(np.mean(bl_low),2), round(np.std(bl_low),2), len(bl_low))
# print("Right Upper Arm: ", round(np.mean(bl_rup),2), round(np.std(bl_rup),2), len(bl_rup))
# print("Right Lower Arm: ", round(np.mean(bl_row),2), round(np.std(bl_row),2), len(bl_row))
# print("Left Abdomen: ", round(np.mean(bl_lab),2), round(np.std(bl_lab),2), len(bl_lab))
# print("Right Abdomen: ", round(np.mean(bl_rab),2), round(np.std(bl_rab),2), len(bl_rab))
# print("Chest: ", round(np.mean(bl_chst),2), round(np.std(bl_chst),2), len(bl_chst))
# print("Beanie 1: ", round(np.mean(bl_b1),2), round(np.std(bl_b1),2), len(bl_b1))
# print("Beanie 2: ", round(np.mean(bl_b2),2), round(np.std(bl_b2),2), len(bl_b2))
# print("Beanie 3: ", round(np.mean(bl_b3),2), round(np.std(bl_b3),2), len(bl_b3))
# print("Beanie Left: ", round(np.mean(bl_bL),2), round(np.std(bl_bL),2), len(bl_bL))
# print("Beanie Right: ", round(np.mean(bl_bR),2), round(np.std(bl_bR),2), len(bl_bR))

In [ ]:
pi_back = 92*[16.6]+14*[15.8]+[18.47]*32+[21.64]*75+[26.23]*39 + 97*[24.54]+[16.96]*176+108*[23.55]+120*[24.9]+281*[20.99]+41*[29.57]
pi_neck = 49*[21.49]+4*[16.97]+[27.79]*7 +[19.39]*21+11*[18.02]+[26.56]*7+92*[19.75]+86*[21.46] + 19*[26.38]+143*[16.42]+165*[22.35]
pi_lup = 303*[18.54]+[17.46]*508+[24.91]*84+87*[24.65] + [18.08]*139+258*[22.1] + 62*[23.43]+47*[18.69]+351*[22.64]+[21.35]*155+176*[24.45]
pi_low = 290*[20.06]+[17.70]*107+[29.35]*31 +90*[23.69]+123*[21.21] + [31.79]*8 + [25.63]*4 + [27.07]*6+ 264*[23.68]+119*[19.66]+7*[26.05]
pi_rup = [18.35]*349+[17.38]*211+90*[27.36] + [15.46]*122 + 400*[18.65]+257*[18.47] + 160*[21.88] + 328*[21.84]+90*[19.77]+303*[19.61]+227*[19.18]
pi_row = [21.00]*66 +[24.59]*17 +48*[24.68]+29*[30.46]+ 65*[24.92] + 19*[25.94]+143*[22.42]+17*[25.56]+43*[9.47]
pi_lab = 20*[28.5]+[24.5]*224 +[17.65]+[14.07]*23+101*[23.34]+11*[19.33]+150*[12.37] + 352*[14.71]+222*[14.58]+384*[13.22]+306*[21.15]
pi_rab = [29.14]*71+[25.68]*103+[11.3]*409 + 121*[21.16]+60*[17.76]+451*[11.8] + [8.44]*685+50*[28.76]+[12.48]*627+3*[9.75]+54*[25.25]
pi_cst = 303*[13.63]+[14.10]*489+[22.69]*196+108*[26.43]+122*[20.85]+ [15.55 ]*161 + [16.59]*482+161*[18.78]+[17.3]*337+4*[14.12]+309*[16.91]
pi_b1 = 42*[22.94]+[17.36]*133 +33*[22.66]+135*[18.93]+[22.62]*75 + 156*[23.04] + 108*[24.41]+18*[25.65]+7*[21.56]+23*[20.83]
pi_b2 = [16.4]*136+[23.88]* 64 +[24.6]*175+41*[20.02]+[24.16]*123 + 201*[22.51] + 301*[19.88]+299*[24.1]+278*[26.25]+24*[22.05]+116*[17.45]
pi_b3 = [22.73]*152+[27.12]*42  +[26.34]*133 +[23.78]*100+[25.03]*117 + 92*[25.49] + [25.73]*263+ 21*[23.27]+97*[15.84]
pi_bL = [17.04]*208 +20*[20.89] + [23.69]*100+56*[27.38]+[25.47]*37 + 32*[22.82]+3*[22.64] + 142*[21.79]
pi_bR = [24.88]*42  +183*[25.52]+[24.3] + 82*[23.52]+49*[26.82] + 100*[23.08]+ 11*[25.16]+[27.02]*267+124*[19.28] + 51*[24.11]

In [ ]:
print("Back: ", round(np.mean(pi_back),2), round(np.std(pi_back),2), len(pi_back))
print("Neck: ", round(np.mean(pi_neck),2), round(np.std(pi_neck),2), len(pi_neck))
print("Left Upper: ", round(np.mean(pi_lup),2), round(np.std(pi_lup),2), len(pi_lup))
print("Left Lower: ", round(np.mean(pi_low),2), round(np.std(pi_low),2), len(pi_low))
print("Right Upper: ", round(np.mean(pi_rup),2), round(np.std(pi_rup),2), len(pi_rup))
print("Right Lower: ", round(np.mean(pi_row),2), round(np.std(pi_row),2), len(pi_row))
print("Left Ab: ", round(np.mean(pi_lab),2), round(np.std(pi_lab),2), len(pi_lab))
print("Right Ab: ", round(np.mean(pi_rab),2), round(np.std(pi_rab),2), len(pi_rab))
print("Chest: ", round(np.mean(pi_cst),2), round(np.std(pi_cst),2), len(pi_cst))
print("Beanie 1: ", round(np.mean(pi_b1),2), round(np.std(pi_b1),2), len(pi_b1))
print("Beanie 2: ", round(np.mean(pi_b2),2), round(np.std(pi_b2),2), len(pi_b2))
print("Beanie 3: ", round(np.mean(pi_b3),2), round(np.std(pi_b3),2), len(pi_b3))
print("Beanie L: ", round(np.mean(pi_bL),2), round(np.std(pi_bL),2), len(pi_bL))
print("Beanie R: ", round(np.mean(pi_bR),2), round(np.std(pi_bR),2), len(pi_bR))

In [ ]:
d4_back = [25.28]*32+61*[16.36]+81*[15.72]+[14.19]*26+5*[20.63]
d4_neck =74*[24.06]+91*[19.36]+47*[20.49]+31*[10.94]+[21.75]*82
d4_lup =[21.22]*259+368*[18.12]+369*[16.01] + [29.39]*571 + 79*[20.37]
d4_low =[18.81]*210+296*[15.54]+98*[19.63]+141*[21.26]+202*[18.08]
d4_rup =[23.24]*116+6*[27.98]+745*[18.53]+380*[17.14]+[25.12]*201+32*[22.63]
d4_row =72*[17.1]+[27.1]+347*[16.51]+174*[16.8]+19*[9.92]+80*[18.68]
d4_lab =101*[19.31]+19*[25.02]+73*[15.37] + 16*[14.28]+[20.0]*26+21*[17.77]
d4_rab =155*[21.21]+[25.3]+397*[15.46]+[12.20]*47+425*[24.46]+72*[20.38]
d4_cst =[19.29]*212+5*[26.17]+320*[16.76]+74*[15.75]+908*[28.84]+144*[21.33]
d4_b1 =[23.95]*96+[19.09]*2+206*[16.40]+[12.75]*125+22*[10.62]+[15.04]*163
d4_b2 =[16.37]*74+260*[22.14]+56*[16.37]+173*[27.12]+[13.91]*168
d4_b3 =[18.16]*40+134*[20.17]+18*[2.93]+956*[26.75]+89*[14.98]
d4_bL =[20.33]*93+102*[17.88]+100*[24.82]+38*[14.15]
d4_bR =29*[21.23]+[18.4]+55*[13.33]+[26.92]*57+86*[15.66]

print("Back: ", round(np.mean(d4_back),2), round(np.std(d4_back),2), len(d4_back))
print("Neck: ", round(np.mean(d4_neck),2), round(np.std(d4_neck),2), len(d4_neck))
print("Left Upper: ", round(np.mean(d4_lup),2), round(np.std(d4_lup),2), len(d4_lup))
print("Left Lower: ", round(np.mean(d4_low),2), round(np.std(d4_low),2), len(d4_low))
print("Right Upper: ", round(np.mean(d4_rup),2), round(np.std(d4_rup),2), len(d4_rup))
print("Right Lower: ", round(np.mean(d4_row),2), round(np.std(d4_row),2), len(d4_row))
print("Left Ab: ", round(np.mean(d4_lab),2), round(np.std(d4_lab),2), len(d4_lab))
print("Right Ab: ", round(np.mean(d4_rab),2), round(np.std(d4_rab),2), len(d4_rab))
print("Chest: ", round(np.mean(d4_cst),2), round(np.std(d4_cst),2), len(d4_cst))
print("Beanie 1: ", round(np.mean(d4_b1),2), round(np.std(d4_b1),2), len(d4_b1))
print("Beanie 2: ", round(np.mean(d4_b2),2), round(np.std(d4_b2),2), len(d4_b2))
print("Beanie 3: ", round(np.mean(d4_b3),2), round(np.std(d4_b3),2), len(d4_b3))
print("Beanie L: ", round(np.mean(d4_bL),2), round(np.std(d4_bL),2), len(d4_bL))
print("Beanie R: ", round(np.mean(d4_bR),2), round(np.std(d4_bR),2), len(d4_bR))

In [ ]:
w_back = [7.9]*70+116*[13.14]+[5.32]* 46+23*[14.47]+[11.51]*16+[9.05]* 14+41*[9.37]+[11.92]* 11+[8.32]* 8+[11.79]* 6+4*[7.37]
w_neck =82*[10.05]+[11.13]* 13+[14.63]*18+[12.7]*54+39*[14.38]+49*[11.15]+[13.27]* 3+[18.33]* 29+[11.3]* 69+[13.45]*21+2*[7.5]
w_lup =[4.33] *61+21*[6.85]+[10.98]*107+138*[7.51]+79*[10.44]+110*[7.91]+[9.16 ]*4+ [8.02 ]*13+[4.74]* 29+[12.77]* 44+3*[8.4]
w_low =3*[8.29]+13*[5.95]+30*[7.77]+51*[7.23]+17*[11.43]+[10.73]* 2+[6.67]* 6+[ 7.43 ]*18+[8.71]* 17
w_rup =[7.1]*30+141*[8.12]+2*[12.29]+43*[6.41]+71*[5.82]+[5.32]* 17+[9.75]+[6.25]* 4+[ 6.34 ]*38+[7.65]* 22+7*[6.47]
w_row =4*[4.46]+29*[5.59]+8*[9.0]+31*[8.94]+[9.79]* 1+[9.1]* 18+[1.32]* 2+[12.57]* 4+[5.7]
w_lab =33*[6.8]+108*[3.62]+200*[4.77]+78*[7.86]+16*[6.26]+[3.69]+[3.16 ]*46+[3.13]*2+16*[5.82]+[6.97]+2*[4.81]
w_rab =5*[-0.21]+[4.99]*100+140*[6.37]+37*[3.10]+[4.25]* 20+[4.83 ]*33+[4.48]+[2.25]* 3+[4.52]* 3
w_cst =18*[11.20]+54*[5.91]+[9.7]+65*[6.83]+[8.21]*7+2*[8.46]+[5.17]* 31+[1.35]* 13+[11.12 ]*6+[9.97]* 6
w_b1 =[5.2]+10*[12.54]+[11.67]*76+3*[12.46]+24*[8.69]+5*[14.79]+18*[12.63]+20*[10.09]+[11.1]* 28+3*[10.79]
w_b2 =47*[15.12]+52*[14.89]+3*[11.53]+7*[13.24]+9*[9.77]+[15.25]* 17+19*[12.2]+[7.26]* 6
w_b3 =58*[12.9]+12*[12.12]+[13.32 ]*8+[14.8]* 8+[18.75]* 14+[14.84]* 17+[9.89]*11+[4.11]* 2
w_bL =6*[14.38]+20*[13.2]+9*[12.01]+[15.42]*2+[16.57]*1+[14.0]*37+3*[16.64]
w_bR =7*[16.6]+263*[17.88]+8*[13.56]+2*[6.42]+7*[18.36]+11*[15.46]+2*[14.65]

print("Back: ", round(np.mean(w_back),2), round(np.std(w_back),2), len(w_back))
print("Neck: ", round(np.mean(w_neck),2), round(np.std(w_neck),2), len(w_neck))
print("Left Up: ", round(np.mean(w_lup),2), round(np.std(w_lup),2), len(w_lup))
print("Right Up: ", round(np.mean(w_rup),2), round(np.std(w_rup),2), len(w_rup))
print("Left Low: ", round(np.mean(w_low),2), round(np.std(w_low),2), len(w_low))
print("Right Low: ", round(np.mean(w_row),2), round(np.std(w_row),2), len(w_row))
print("Left Ab: ", round(np.mean(w_lab),2), round(np.std(w_lab),2), len(w_lab))
print("Right A: ", round(np.mean(w_rab),2), round(np.std(w_rab),2), len(w_rab))
print("Chest: ", round(np.mean(w_cst),2), round(np.std(w_cst),2), len(w_cst))
print("Beanie 1: ", round(np.mean(w_b1),2), round(np.std(w_b1),2), len(w_b1))
print("Beanie 2: ", round(np.mean(w_b2),2), round(np.std(w_b2),2), len(w_b2))
print("Beanie 3: ", round(np.mean(w_b3),2), round(np.std(w_b3),2), len(w_b3))
print("Beanie L: ", round(np.mean(w_bL),2), round(np.std(w_bL),2), len(w_bL))
print("Beanie R: ", round(np.mean(w_bR),2), round(np.std(w_bR),2), len(w_bR))